In [1]:
import napari
import numpy as np
from skimage import data

import napari_plot
from napari_plot._qt.qt_viewer import QtViewer
from dask_image.imread import imread
from magicgui.widgets import ComboBox, Container, PushButton, SpinBox, FileEdit

from dask_image import imread
import pandas as pd
import dask.array as da
import os
import dask.dataframe as dd

C:\Users\pierc\Anaconda3\envs\Pytorch\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [2]:
class neuro_viewer(object):
    
    im = None
    denoised_dir = None
    volume_df = None
    coms_df = None
    con_df = None
    dff_tidy = None
    
    
    def __init__(self, fr = 3.07):
        
        

        self.viewer = napari.Viewer()
        self.viewer.add_image(np.zeros((512, 512)))
        self.im_subset = self.viewer.layers[0]
        self.im_subset.colormap = "inferno"
        self.stim_icon = self.viewer.add_shapes(np.array([[0, 0, 0],
                                                           [0, 0, 0],
                                                           [0, 0, 0],
                                                           [0, 0, 0]]), shape_type='rectangle', edge_width=5,
                                                            edge_color='#55ff00', face_color = "#55ff00")
        #self.all_rois = self.viewer.add_shapes(shape_type = "polygon", edge_width=0.2,
        #                  edge_color='white', face_color='transparent', ndim = 4)
        self.select_roi = self.viewer.add_shapes(shape_type='polygon', edge_width=0.2,
                          edge_color='white', face_color='transparent', ndim = 4)
        
        self.im_subset.contrast_limits = (0, 10000)
        self.fr = fr
        
        
        # create label menus for stim, trial, angle, velocity
        #update choices as required when stim is changed
        self.stim_menu = ComboBox(label='Stim', choices = ["visual", "motion", "flow"], tooltip = "Select stim")
        self.trial_menu = ComboBox(label='Trial', choices = [], tooltip = "Select stim")
        self.angle_menu = ComboBox(label='Angle', choices = [], tooltip = "Select stim")
        self.velocity_menu = ComboBox(label='Velocity', choices = [], tooltip = "Select stim")
        
        
        dropdown_widget = Container(widgets=[self.stim_menu, self.trial_menu, self.angle_menu, self.velocity_menu])
        
        
        self.stim_menu.changed.connect(self.stim_changed)
        self.trial_menu.changed.connect(self.trial_changed)
        self.angle_menu.changed.connect(self.angle_changed)
        self.velocity_menu.changed.connect(self.velocity_changed)
        
        
        
        
        self.cell_ids = SpinBox(label = "cell_id", tooltip = "change cell")
        cell_widget = Container(widgets=[self.cell_ids])
        
        self.volume_df_picker= FileEdit(value='./Select volume_df.h5', tooltip = "Select volume_df")
        
        file_widget = Container(widgets=[self.volume_df_picker])
        self.volume_df_picker.changed.connect(self.volume_df_picker_changed)
        self.viewer.window.add_dock_widget(dropdown_widget)
        self.viewer.window.add_dock_widget(cell_widget)
        self.viewer.window.add_dock_widget(file_widget)
        self.add_dff_widget()
        self.cell = self.cell_ids.value
        self.cell_ids.changed.connect(self.plot_cell)
        
        
        
        
        
        
        """push_button = PushButton(label = "Save Labels")
        start_button = PushButton(label = "First keypoint")
        end_button = PushButton(label = "Last keypoint")

        
        self.ind_spinbox = SpinBox(label = "Individual Number", tooltip = "Change individual")
        label_txt_picker = FileEdit(value='./Select a labeled txt file', tooltip = "Select labeled txt file")
        label_h5_picker = FileEdit(value='./Select a labeled h5 file', tooltip = "Select labeled h5 file")
        h5_picker = FileEdit(value='./Select a DLC h5 file', tooltip = "Select h5 file")
        vid_picker = FileEdit(value='./Select the corresponding raw video', tooltip = "Select corresponding raw video")
        file_widget = Container(widgets=[label_h5_picker, h5_picker, vid_picker, label_txt_picker])
        label_widget = Container(widgets=[self.label_menu])
        class_widget = Container(widgets=[self.ind_spinbox, self.spinbox, push_button, start_button, end_button])
        # add individual number spinbox (set limits based on number of indivduals)
        # each ind should have a dictionary containing sub behaviour dictionaries storing coords and label

        self.viewer.window.add_dock_widget(file_widget)
        self.viewer.window.add_dock_widget(label_widget)
        self.viewer.window.add_dock_widget(class_widget)

        # add file widget for h5 and video
        self.labeled_txt = label_txt_picker.value
        self.labeled_h5 = label_h5_picker.value
        self.h5_file = h5_picker.value
        self.video_file = vid_picker.value"""
        
    def load_image(self):
        """Load image"""
        if self.volume_subset.shape[0] > 0:
            print("loading dask images")
            self.im_subset.data = self.im[:, self.vols].compute(scheduler='processes')
            
        else:
            print("no volumes")
        pass
    
    def load_volume_df(self, path):
        """Load pandas dataframe containing all information about volume"""
        print("loading volume_df")
        self.volume_df = pd.read_hdf(path)
        
    def load_coms_df(self, path):
        print("loading coms_df")
        self.coms_df = pd.read_hdf(path)
        
    
    def load_con_df(self, path):
        print("loading con_df")
        self.con_df = pd.read_hdf(path)
        
    def load_dff_df(self, path):
        """Load all dff data"""
        print("loading dff_df")
        self.dff_tidy = pd.read_hdf(path, "df")
    
    def lazy_load_images(self):
        
        print("loading images")
        # load all images as one dask array - dask.array.stack(data = [],  axis = 0)
        files = os.listdir(self.denoised_dir)
        ims = [os.path.join(self.denoised_dir,file) for file in files if 'denoised' in file]
        da_ims = []

        for im in ims:
            da_im = imread.imread(im)
            da_ims.append(da_im)
            
        self.im = da.stack(da_ims, axis = 0)
        print("loading finished")
        
    
    def stim_changed(self, event):
        print(self.volume_df.trial.unique())
        self.stim = event
        self.stim_subset  = self.volume_df[self.volume_df.stim == self.stim]
        self.update_menu_choices()
        self.load_subset()
        print(event)
        
    def load_subset(self):
        self.get_volume_subset()
        if self.volume_subset.shape[0] > 0:
            self.load_image()
            self.plot_cell(self.cell)
            self.label_stim()
            
        
        
    def update_menu_choices(self):
        trials = self.stim_subset.trial.dropna().unique().tolist()
        angles = self.stim_subset.angle.dropna().unique().tolist()
        velocities = self.stim_subset.velocity.dropna().unique().tolist()
        
        self.angle = angles[0]
        self.velocity = velocities[0]
        self.trial = trials[0]
        
        # initiates change loop
        self.trial_menu.choices = trials
        self.angle_menu.choices = angles
        self.velocity_menu.choices = velocities
        
        
        print(self.angle, self.velocity, self.trial)
        # update choices of trial, angle, velocity
        
        
    def trial_changed(self, event):
        self.trial = event
        self.load_subset()
        print("new trial {}".format(event))
        
    def angle_changed(self, event):
        self.angle = event
        self.load_subset()
        
        print("new angle {}".format(event))
        
    def velocity_changed(self, event):
        self.velocity = event
        self.load_subset()
        print("new velocity {}".format(event))
        
    def volume_df_picker_changed(self, event):
        print(event)
        
        try:
            self.volume_df_path = event
        except:
            self.volume_df_path = event.value
        
        
        # get denoised dir
        self.denoised_dir = os.path.dirname(self.volume_df_path)
        
        # load all dataframes
        
        self.load_volume_df(self.volume_df_path)
        self.load_coms_df(os.path.join(self.denoised_dir, "all_neuron_centers.h5"))
        self.load_con_df(os.path.join(self.denoised_dir, "all_neuron_contours.h5"))
        #self.load_dff_df(os.path.join(self.denoised_dir, "dff.h5"))
        self.lazy_load_images()
        
        self.stim_changed(self.stim_menu.value)
        
        
    def get_volume_subset(self):
        print(self.stim, self.angle, self.velocity, self.trial)
        stim_filter = self.volume_df.stim == self.stim
        angle_filter = self.volume_df.angle == self.angle
        velocity_filter = self.volume_df.velocity == self.velocity
        trial_filter = self.volume_df.trial == self.trial
        
        self.volume_subset = self.volume_df[(stim_filter) & (angle_filter) & ( velocity_filter) & (trial_filter)]
        
        if self.volume_subset.shape[0] > 0:
         # 4 seconds before
            vols_before = int(4 * self.fr)
            
            self.cond_start  = self.volume_subset.nVol.iloc[0] 
            self.cond_end = self.volume_subset.nVol.iloc[-1] 
            self.first_vol = self.cond_start - vols_before
            self.end_vol = self.cond_end + vols_before
            self.vols = np.arange(self.first_vol, self.end_vol)
            
    def add_dff_widget(self):
        
        self.viewer1d = napari_plot.ViewerModel1D()
        widget = QtViewer(self.viewer1d)
        self.viewer.window.add_dock_widget(widget, area="bottom", name="Line Widget")
        self.viewer1d.axis.x_label = "Time"
        self.viewer1d.axis.y_label = "DeltaF/F"
        self.viewer1d.reset_view()
        
    def clear_plot(self):
        self.viewer1d.clear_canvas()
        
        
    def plot_cell(self, event):
        
        self.clear_plot()
        # define cell event
        # get dff info for cell
        # get contour of cell
        self.cell = event
        
        stim_filter = self.dff_tidy.stim == self.stim
        angle_filter = self.dff_tidy.angle == self.angle
        velocity_filter = self.dff_tidy.velocity == self.velocity
        trial_filter = self.dff_tidy.trial == self.trial
        cell_filter = self.dff_tidy.cell_id == self.cell
        
        
        self.dff_subset  = self.dff_tidy[(stim_filter) & (angle_filter) & (velocity_filter) & (cell_filter)]
        
        # plot every trial but highlight current one with thicker line
        
        for trial in self.dff_subset.trial.unique():
            trial_subset = self.dff_subset[self.dff_subset.trial == trial]
            trial_start = trial_subset.nVol.iloc[0]
            trial_end = trial_subset.nVol.iloc[-1]
            vols_before = int(4 * self.fr)
            first_vol = trial_start - vols_before
            last_vol = trial_end + vols_before
            vols = np.arange(first_vol, last_vol)
            vol_filter = self.dff_tidy.nVol.isin(vols)

            plotting_subset = self.dff_tidy[cell_filter & vol_filter]
            t = np.arange(0, plotting_subset.shape[0]/self.fr, 1/self.fr)

            if trial == self.trial:

                self.viewer1d.add_line(np.c_[t, plotting_subset.dff.to_numpy()], color = "magenta")
                regions = [
                        ([t[trial_start-first_vol], t[trial_end-first_vol]], "vertical"),
                    ]

                layer = neuroviewer.viewer1d.add_region(
                        regions,
                        color=["green"],
                        opacity = 0.2,
                        name = "Stim",
                    )

            else:
                self.viewer1d.add_line(np.c_[t, plotting_subset.dff.to_numpy()], name="trial {}".format(trial), color="gray")
                
           

            self.viewer1d.reset_view()
            self.plot_cell_contour()
        
    def label_stim(self):
        start = self.cond_start - self.first_vol
        end = self.cond_end - self.first_vol
        stim_duration = end - start

        nframes = stim_duration
        centre_rs = np.tile(np.array([0, 30, 30]), (4, 1))
        centres = np.stack([centre_rs] * nframes)
        frame_pos = np.tile(np.arange(start, end).T, (4, 1)).T
        centres[:, :, 0] = frame_pos
        add_array = np.array([[0, -10, -10],
                                      [0, -10, 10],
                                      [0, 10, 10],
                                      [0, 10, -10]])

        #define boxes by adding to centre_rs
        boxes = centres + add_array.reshape(-1, *add_array.shape)
        labels = ["stim"] * stim_duration
        properties = {
            'label': labels,

        }

        # specify the display parameters for the text
        text_params = {
            'text': 'label: {label}',
            'size': 12,
            'color': 'green',
            'anchor': 'upper_left',
            'translation': [-3, 0]}

        self.stim_icon.data = boxes
        self.stim_icon.properties = properties
        self.stim_icon.text = text_params
        
    def plot_cell_contour(self): # slow
        contours = self.con_df[self.con_df.cell_id == self.cell].dropna()
        contours = contours[["z","cell_id", "y", "x"]]

        contours.cell_id = np.zeros(contours.shape[0])
        cell_roi = contours.to_numpy()
        start = self.first_vol
        end = self.end_vol
        nframes = end - start

        cell_rois = np.stack([cell_roi] * nframes)
        t_index = cell_rois[:, :, 1]
        new_t_index = np.tile(np.arange(t_index.shape[0]), (t_index.shape[1], 1)).T
        cell_rois[:, :, 1] = new_t_index
        self.select_roi.selected_data = set(range(neuroviewer.select_roi.nshapes))
        self.select_roi.remove_selected()

        self.select_roi.add(cell_rois,shape_type = "polygon", edge_width=0.2,
                                  edge_color='white', face_color='transparent')


        
        
            

    
        
        
    
        
    

In [3]:
neuroviewer = neuro_viewer()
# to do - extend window to 4 seconds before and after
# plot dff in bit below

C:\Users\pierc\Anaconda3\envs\Pytorch\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [4]:
neuroviewer.load_dff_df(r"C:\Users\pierc\Desktop\dff.h5")

loading dff_df
Z:\Pierce\IOMultiSensory\20220420\fish1\denoised\volume_df.h5
loading volume_df
loading coms_df
loading con_df
loading images
loading finished
[nan  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
visual 225.0 0.0 0.0
new trial 0.0
visual 225.0 0.0 0.0
new angle 225.0
visual 225.0 0.0 0.0
new velocity 0.0
225.0 0.0 0.0
visual 225.0 0.0 0.0
visual
visual 225.0 1.0 0.0
loading dask images


In [20]:
neuroviewer.plot_cell(0)

In [ ]:
centre_rs = np.tile(np.array([0, 30, 30]), (4, 1))
centres = np.stack([centre_rs] * nframes)
frame_pos = np.tile(np.arange(start, end).T, (4, 1)).T

In [30]:
neuroviewer.volume_subset.shape

(14, 12)

In [25]:
# loop through cells get x, y contour for each #
# add first axis for volumes
# get z_index
start = neuroviewer.first_vol
end = neuroviewer.end_vol
nframes = end - start


z_index = [] 
shapes= []
for cell in neuroviewer.con_df.cell_id.unique():
    subset = neuroviewer.con_df[neuroviewer.con_df.cell_id == cell].dropna()
    contours = subset[["z","cell_id", "y", "x"]]
    
    contours.cell_id = np.zeros(contours.shape[0])

    
    z_index.append(subset.iloc[0, 2])
    shapes.append(contours.to_numpy())
    

In [171]:
len(z_index), len(shapes)

(467, 467)

In [108]:
neuroviewer.viewer.add_shapes([cell_roi], shape_type='polygon', edge_width=0.2,
                          edge_color='white', face_color='transparent', ndim = 4)

<Shapes layer 'Shapes [3]' at 0x1f76e416250>

In [112]:
neuroviewer.select_roi = neuroviewer.viewer.layers[-1]

In [110]:
neuroviewer.viewer.layers[-1]

<Shapes layer 'Shapes [3]' at 0x1f76e416250>

In [153]:
cell_roi

array([[  0.        ,   0.        , 129.14676135, 163.        ],
       [  0.        ,   0.        , 130.        , 162.77276648],
       [  0.        ,   0.        , 130.82616582, 162.        ],
       [  0.        ,   0.        , 131.        , 161.79858813],
       [  0.        ,   0.        , 131.45820887, 161.        ],
       [  0.        ,   0.        , 131.81591936, 160.        ],
       [  0.        ,   0.        , 132.        , 159.5665173 ],
       [  0.        ,   0.        , 132.40061053, 159.        ],
       [  0.        ,   0.        , 132.71985199, 158.        ],
       [  0.        ,   0.        , 132.68592001, 157.        ],
       [  0.        ,   0.        , 132.5509626 , 156.        ],
       [  0.        ,   0.        , 132.        , 155.20787663],
       [  0.        ,   0.        , 131.77302779, 155.        ],
       [  0.        ,   0.        , 131.        , 154.19547376],
       [  0.        ,   0.        , 130.78387183, 154.        ],
       [  0.        ,   0

In [137]:
neuroviewer.select_roi.add(cell_roi)

[[  0.           0.         173.97279412 144.        ]
 [  0.           0.         174.         143.96836888]
 [  0.           0.         174.8705369  143.        ]
 [  0.           0.         174.79064643 142.        ]
 [  0.           0.         174.         141.16975418]
 [  0.           0.         173.88571776 141.        ]
 [  0.           0.         173.         140.09879453]
 [  0.           0.         172.4038055  140.        ]
 [  0.           0.         172.         139.77996208]
 [  0.           0.         171.         139.88615078]
 [  0.           0.         170.86126303 140.        ]
 [  0.           0.         170.04903623 141.        ]
 [  0.           0.         170.         141.01569467]
 [  0.           0.         169.49246518 142.        ]
 [  0.           0.         169.59797756 143.        ]
 [  0.           0.         170.         143.62771297]
 [  0.           0.         170.22394619 144.        ]
 [  0.           0.         171.         144.75887664]
 [  0.    

ValueError: Data shape does not match a rectangle. Rectangle expects four corner vertices, 21 provided.

In [97]:
cell_roi.shape

(65, 4)

In [145]:
neuroviewer.select_roi.data

[array([[  0.        ,   0.        , 173.97279412, 144.        ],
        [  0.        ,   0.        , 174.        , 143.96836888],
        [  0.        ,   0.        , 174.8705369 , 143.        ],
        [  0.        ,   0.        , 174.79064643, 142.        ],
        [  0.        ,   0.        , 174.        , 141.16975418],
        [  0.        ,   0.        , 173.88571776, 141.        ],
        [  0.        ,   0.        , 173.        , 140.09879453],
        [  0.        ,   0.        , 172.4038055 , 140.        ],
        [  0.        ,   0.        , 172.        , 139.77996208],
        [  0.        ,   0.        , 171.        , 139.88615078],
        [  0.        ,   0.        , 170.86126303, 140.        ],
        [  0.        ,   0.        , 170.04903623, 141.        ],
        [  0.        ,   0.        , 170.        , 141.01569467],
        [  0.        ,   0.        , 169.49246518, 142.        ],
        [  0.        ,   0.        , 169.59797756, 143.        ],
        [ 

In [26]:

#neuroviewer.all_rois.data = shapes
#neuroviewer.all_rois.edge_color = "white"
#neuroviewer.all_rois.face_color = "transparent"

neuroviewer.viewer.add_shapes(shapes, shape_type='polygon', edge_width=0.2,
                          edge_color='white', face_color='transparent', z_index = z_index)

AxisError: axis 1 is out of bounds for array of dimension 0

In [112]:


stim_duration

13

In [99]:
start =12
end = 24
stim_duration = end - start

In [121]:
start = neuroviewer.cond_start - neuroviewer.first_vol
end = neuroviewer.cond_end - neuroviewer.first_vol
stim_duration = end - start

nframes = stim_duration
centre_rs = np.tile(np.array([0, 30, 30]), (4, 1))
centres = np.stack([centre_rs] * nframes)
frame_pos = np.tile(np.arange(start, end).T, (4, 1)).T
centres[:, :, 0] = frame_pos
add_array = np.array([[0, -10, -10],
                              [0, -10, 10],
                              [0, 10, 10],
                              [0, 10, -10]])

#define boxes by adding to centre_rs
boxes = centres + add_array.reshape(-1, *add_array.shape)
labels = ["stim"] * stim_duration
properties = {
    'label': labels,

}

# specify the display parameters for the text
text_params = {
    'text': 'label: {label}',
    'size': 12,
    'color': 'green',
    'anchor': 'upper_left',
    'translation': [-3, 0]}

neuroviewer.stim_icon.data = boxes
neuroviewer.stim_icon.properties = properties
neuroviewer.stim_icon.text = text_params



In [124]:
neuroviewer.stim_icon.data

[array([[12., 20., 20.],
        [12., 20., 40.],
        [12., 40., 40.],
        [12., 40., 20.]]),
 array([[13., 20., 20.],
        [13., 20., 40.],
        [13., 40., 40.],
        [13., 40., 20.]]),
 array([[14., 20., 20.],
        [14., 20., 40.],
        [14., 40., 40.],
        [14., 40., 20.]]),
 array([[15., 20., 20.],
        [15., 20., 40.],
        [15., 40., 40.],
        [15., 40., 20.]]),
 array([[16., 20., 20.],
        [16., 20., 40.],
        [16., 40., 40.],
        [16., 40., 20.]]),
 array([[17., 20., 20.],
        [17., 20., 40.],
        [17., 40., 40.],
        [17., 40., 20.]]),
 array([[18., 20., 20.],
        [18., 20., 40.],
        [18., 40., 40.],
        [18., 40., 20.]]),
 array([[19., 20., 20.],
        [19., 20., 40.],
        [19., 40., 40.],
        [19., 40., 20.]]),
 array([[20., 20., 20.],
        [20., 20., 40.],
        [20., 40., 40.],
        [20., 40., 20.]]),
 array([[21., 20., 20.],
        [21., 20., 40.],
        [21., 40., 40.],
       

visual 0.0 1.0 0.0
loading dask images
new angle 0.0


In [18]:
dff = pd.read_hdf(r"C:\Users\pierc\Desktop\dff.h5")
dff.head()

,nVol,cell_id,dff,angle,velocity,trial,stim,time,timebyepoch,Rightpower,Leftpower,Leftburst_rate,Rightburst_rate,Rightconvolved,Leftconvolved
0,0,0,0.339105,NaN,0.0,NaN,visual,0.000000,0.000000,0.019171,0.011254,0.043933,0.000469,0.150466,0.084270
1,1,0,0.390035,NaN,0.0,NaN,visual,0.325733,0.325733,0.019783,0.011051,0.044881,0.000535,0.293180,0.189696
2,2,0,0.655681,NaN,0.0,NaN,visual,0.651466,0.651466,0.018639,0.010802,0.045771,0.000608,0.169023,0.069403
3,3,0,0.533068,NaN,0.0,NaN,visual,0.977199,0.977199,0.019333,0.011246,0.046599,0.000690,0.280450,0.186175
4,4,0,0.542190,NaN,0.0,NaN,visual,1.302932,1.302932,0.019168,0.011754,0.047361,0.000782,0.170353,0.073408


In [26]:
neuroviewer.dff_tidy.head()

,nVol,cell_id,dff,angle,velocity,trial,stim,time,timebyepoch,Rightpower,Leftpower,Leftburst_rate,Rightburst_rate,Rightconvolved,Leftconvolved
0,0,0,0.339105,NaN,0.0,NaN,visual,0.000000,0.000000,0.019171,0.011254,0.043933,0.000469,0.150466,0.084270
1,1,0,0.390035,NaN,0.0,NaN,visual,0.325733,0.325733,0.019783,0.011051,0.044881,0.000535,0.293180,0.189696
2,2,0,0.655681,NaN,0.0,NaN,visual,0.651466,0.651466,0.018639,0.010802,0.045771,0.000608,0.169023,0.069403
3,3,0,0.533068,NaN,0.0,NaN,visual,0.977199,0.977199,0.019333,0.011246,0.046599,0.000690,0.280450,0.186175
4,4,0,0.542190,NaN,0.0,NaN,visual,1.302932,1.302932,0.019168,0.011754,0.047361,0.000782,0.170353,0.073408


visual 225.0 1.0 0.0
new velocity 1.0
visual 0.0 1.0 0.0
new angle 0.0
visual 0.0 1.0 1.0
new trial 1.0


In [6]:
stim_filter = neuroviewer.dff_tidy.stim == neuroviewer.stim
angle_filter = neuroviewer.dff_tidy.angle == neuroviewer.angle
velocity_filter = neuroviewer.dff_tidy.velocity == neuroviewer.velocity
trial_filter = neuroviewer.dff_tidy.trial == neuroviewer.trial
cell_filter = neuroviewer.dff_tidy.cell_id == neuroviewer.cell

In [5]:
dff_df = neuroviewer.dff_tidy

visual 225.0 1.0 0.0
new velocity 1.0


In [47]:
#neuroviewer.dff_tidy == dff_df
vol_filter = neuroviewer.dff_tidy.nVol.isin(neuroviewer.vols)
vol_filter

0          False
1          False
2          False
3          False
4          False
           ...  
5571772    False
5571773    False
5571774    False
5571775    False
5571776    False
Name: nVol, Length: 5571777, dtype: bool

In [21]:
regions = [
    ([0, 1], "vertical"),
]

layer = neuroviewer.viewer1d.add_region(
    regions,
    color=[white],
    opacity = 0.5,
    name = "Stim",
)

C:\Users\pierc\Anaconda3\envs\Pytorch\lib\site-packages\napari\layers\utils\color_transformations.py:53: UserWarning: The provided color parameter has 3 entries, while the data contains 2 entries. Setting color to white.
  warnings.warn(


In [7]:
neuroviewer.dff_subset = neuroviewer.dff_tidy[stim_filter & angle_filter & velocity_filter & cell_filter]
neuroviewer.dff_subset.head()

,nVol,cell_id,dff,angle,velocity,trial,stim,time,timebyepoch,Rightpower,Leftpower,Leftburst_rate,Rightburst_rate,Rightconvolved,Leftconvolved
105,105,0,-0.028433,225.0,1.0,0.0,visual,34.201954,0.000000,0.021072,0.010453,0.000037,0.065900,0.247350,0.147832
106,106,0,-0.081905,225.0,1.0,0.0,visual,34.527687,0.325733,0.020749,0.011488,0.000032,0.063622,0.222778,0.115152
107,107,0,0.024480,225.0,1.0,0.0,visual,34.853420,0.651466,0.020536,0.012242,0.000027,0.061332,0.250507,0.144951
108,108,0,-0.005002,225.0,1.0,0.0,visual,35.179153,0.977199,0.019440,0.011264,0.000023,0.059038,0.224487,0.120019
109,109,0,-0.050085,225.0,1.0,0.0,visual,35.504886,1.302932,0.018804,0.011541,0.000019,0.056748,0.246967,0.148896


In [56]:
neuroviewer.dff_subset.shape

(71, 15)

In [78]:
np.arange(0, plotting_subset.shape[0]/neuroviewer.fr, 1/neuroviewer.fr)

array([ 0.        ,  0.3257329 ,  0.6514658 ,  0.9771987 ,  1.3029316 ,
        1.6286645 ,  1.95439739,  2.28013029,  2.60586319,  2.93159609,
        3.25732899,  3.58306189,  3.90879479,  4.23452769,  4.56026059,
        4.88599349,  5.21172638,  5.53745928,  5.86319218,  6.18892508,
        6.51465798,  6.84039088,  7.16612378,  7.49185668,  7.81758958,
        8.14332248,  8.46905537,  8.79478827,  9.12052117,  9.44625407,
        9.77198697, 10.09771987, 10.42345277, 10.74918567, 11.07491857,
       11.40065147, 11.72638436, 12.05211726])

In [76]:
plotting_subset.shape

(38, 15)

array([ 0.        ,  0.25260919,  0.50521837,  0.75782756,  1.01043675,
        1.26304593,  1.51565512,  1.76826431,  2.0208735 ,  2.27348268,
        2.52609187,  2.77870106,  3.03131024,  3.28391943,  3.53652862,
        3.7891378 ,  4.04174699,  4.29435618,  4.54696537,  4.79957455,
        5.05218374,  5.30479293,  5.55740211,  5.8100113 ,  6.06262049,
        6.31522967,  6.56783886,  6.82044805,  7.07305724,  7.32566642,
        7.57827561,  7.8308848 ,  8.08349398,  8.33610317,  8.58871236,
        8.84132154,  9.09393073,  9.34653992,  9.59914911,  9.85175829,
       10.10436748, 10.35697667, 10.60958585, 10.86219504, 11.11480423,
       11.36741341, 11.6200226 , 11.87263179, 12.12524098, 12.37785016])

In [79]:
t.shape

(37,)

In [11]:
for trial in neuroviewer.dff_subset.trial.unique():
    trial_subset = neuroviewer.dff_subset[neuroviewer.dff_subset.trial == trial]
    trial_start = trial_subset.nVol.iloc[0]
    trial_end = trial_subset.nVol.iloc[-1]
    vols_before = int(4 * neuroviewer.fr)
    first_vol = trial_start - vols_before
    last_vol = trial_end + vols_before
    vols = np.arange(first_vol, last_vol)
    vol_filter = neuroviewer.dff_tidy.nVol.isin(vols)
    
    plotting_subset = neuroviewer.dff_tidy[cell_filter & vol_filter]
    t = np.arange(0, plotting_subset.shape[0]/neuroviewer.fr, 1/neuroviewer.fr)

    if trial == neuroviewer.trial:
        
        neuroviewer.viewer1d.add_line(np.c_[t, plotting_subset.dff.to_numpy()], color = "magenta")

    else:
        neuroviewer.viewer1d.add_line(np.c_[t, plotting_subset.dff.to_numpy()], name="trial {}".format(trial), color="gray")
        
    neuroviewer.viewer1d.reset_view()

In [9]:
plotting_subset.shape, t.shape

((38, 15), (37,))

In [61]:
trial_start = trial_subset.nVol.iloc[0]
trial_end = trial_subset.nVol.iloc[-1]
vols_before = int(4 * neuroviewer.fr)
first_vol = trial_start - vols_before
last_vol = trial_end + vols_before
vols = np.arange(first_vol, last_vol)
vol_filter = neuroviewer.dff_tidy.nVol.isin(vols)

plotting_subset = neuroviewer.dff_tidy[cell_filter & vol_filter]

In [64]:
plotting_subset

,nVol,cell_id,dff,angle,velocity,trial,stim,time,timebyepoch,Rightpower,Leftpower,Leftburst_rate,Rightburst_rate,Rightconvolved,Leftconvolved
93,93,0,0.173491,NaN,0.0,0.0,visual,30.293160,11.726384,0.021078,0.012007,0.000215,0.089337,0.242151,0.142579
94,94,0,0.171182,NaN,0.0,0.0,visual,30.618893,12.052117,0.020765,0.011267,0.000187,0.087836,0.218395,0.116583
95,95,0,0.145756,NaN,0.0,0.0,visual,30.944625,12.377850,0.021643,0.010518,0.000163,0.086228,0.253216,0.146234
96,96,0,0.043309,NaN,0.0,0.0,visual,31.270358,12.703583,0.020897,0.011633,0.000142,0.084519,0.228248,0.113423
97,97,0,0.034386,NaN,0.0,0.0,visual,31.596091,13.029316,0.020189,0.011342,0.000123,0.082717,0.257821,0.146698
98,98,0,0.011220,NaN,0.0,0.0,visual,31.921824,13.355049,0.020256,0.012662,0.000106,0.080829,0.225920,0.117831
99,99,0,-0.037694,NaN,0.0,0.0,visual,32.247557,13.680782,0.021304,0.011924,0.000092,0.078864,0.255359,0.154110
100,100,0,0.014465,NaN,0.0,0.0,visual,32.573290,14.006515,0.019730,0.011784,0.000079,0.076829,0.229683,0.124879
101,101,0,-0.071139,NaN,0.0,0.0,visual,32.899023,14.332248,0.021120,0.010930,0.000068,0.074733,0.250855,0.151403
102,102,0,-0.041577,NaN,0.0,0.0,visual,33.224756,14.657980,0.019859,0.011364,0.000059,0.072583,0.224362,0.120377


In [58]:
np.c_[trial_subset.timebyepoch.to_numpy(), trial_subset.dff.to_numpy()]

array([[ 0.00000000e+00, -4.78843896e-02],
       [ 3.25732899e-01,  3.15092933e-02],
       [ 6.51465798e-01, -1.28800987e-01],
       [ 9.77198697e-01,  0.00000000e+00],
       [ 1.30293160e+00, -9.06047163e-02],
       [ 1.62866450e+00, -5.00165820e-02],
       [ 1.95439739e+00, -1.44255206e-02],
       [ 2.28013029e+00,  1.89387312e-02],
       [ 2.60586319e+00,  2.65172577e-02],
       [ 2.93159609e+00, -5.34754528e-03],
       [ 3.25732899e+00, -1.83494460e-03],
       [ 3.58306189e+00, -3.15075723e-02],
       [ 3.90879479e+00, -1.51197592e-02],
       [ 4.23452769e+00, -1.05336424e-01],
       [ 4.56026059e+00, -1.37097642e-01]])

In [41]:
trial_subset.timebyepoch.to_numpy()

array([0.        , 0.3257329 , 0.6514658 , 0.9771987 , 1.3029316 ,
       1.6286645 , 1.95439739, 2.28013029, 2.60586319, 2.93159609,
       3.25732899, 3.58306189, 3.90879479, 4.23452769, 4.56026059])

In [18]:
trial_subset.dff.to_numpy()

array([-0.02843321, -0.08190488,  0.02447959, -0.00500185, -0.05008466,
       -0.06072326, -0.03138618,  0.04699198,  0.06993566,  0.08026482,
        0.09843383,  0.10433007, -0.06768305,  0.02190483])

In [46]:
neuroviewer.velocity = 1.0

stim_filter = neuroviewer.volume_df.stim == neuroviewer.stim
angle_filter = neuroviewer.volume_df.angle == neuroviewer.angle
velocity_filter = neuroviewer.volume_df.velocity == neuroviewer.velocity
trial_filter = neuroviewer.volume_df.trial == neuroviewer.trial

neuroviewer.volume_subset = neuroviewer.volume_df[(stim_filter) & (angle_filter) & ( velocity_filter) & (trial_filter)]
neuroviewer.vols = neuroviewer.volume_subset.nVol
neuroviewer.vols

105    105
106    106
107    107
108    108
109    109
110    110
111    111
112    112
113    113
114    114
115    115
116    116
117    117
118    118
Name: nVol, dtype: int32

float

In [57]:
neuroviewer.trial = 0 

In [39]:
neuroviewer.volume_df[(neuroviewer.volume_df.stim == self.stim)]

,starts,ends,nVol,angle,velocity,stim,x_move,y_move,movingVol,correction_x,correction_y,trial
0,6.201509,6.526009,0,NaN,0.0,visual,NaN,NaN,NaN,NaN,NaN,NaN
1,6.527176,6.851676,1,NaN,0.0,visual,NaN,NaN,NaN,NaN,NaN,NaN
2,6.852676,7.177342,2,NaN,0.0,visual,NaN,NaN,NaN,NaN,NaN,NaN
3,7.178343,7.502842,3,NaN,0.0,visual,NaN,NaN,NaN,NaN,NaN,NaN
4,7.503843,7.828509,4,NaN,0.0,visual,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
945,313.235863,313.560363,11926,NaN,0.0,flow,NaN,NaN,NaN,NaN,NaN,9.0
946,313.561363,313.886029,11927,NaN,0.0,flow,NaN,NaN,NaN,NaN,NaN,9.0
947,313.887029,314.211696,11928,NaN,0.0,flow,NaN,NaN,NaN,NaN,NaN,9.0
948,314.212696,314.537363,11929,NaN,0.0,flow,NaN,NaN,NaN,NaN,NaN,9.0


In [34]:
dff_df = pd.read_hdf(os.path.join(neuroviewer.denoised_dir, "dff.h5"))

In [35]:
dff_df.set_index("cell_id", inplace = True)
dff_df.head()

,nVol,dff,angle,velocity,trial,stim,time,timebyepoch,Rightpower,Leftpower,Leftburst_rate,Rightburst_rate,Rightconvolved,Leftconvolved
cell_id,,,,,,,,,,,,,,
0,0,0.339105,NaN,0.0,NaN,visual,0.000000,0.000000,0.019171,0.011254,0.043933,0.000469,0.150466,0.084270
0,1,0.390035,NaN,0.0,NaN,visual,0.325733,0.325733,0.019783,0.011051,0.044881,0.000535,0.293180,0.189696
0,2,0.655681,NaN,0.0,NaN,visual,0.651466,0.651466,0.018639,0.010802,0.045771,0.000608,0.169023,0.069403
0,3,0.533068,NaN,0.0,NaN,visual,0.977199,0.977199,0.019333,0.011246,0.046599,0.000690,0.280450,0.186175
0,4,0.542190,NaN,0.0,NaN,visual,1.302932,1.302932,0.019168,0.011754,0.047361,0.000782,0.170353,0.073408


In [36]:
neuroviewer.dff_tidy.T.to_hdf(os.path.join(neuroviewer.denoised_dir, "dff_test.h5"), "df")

In [5]:
neuroviewer.dff_tidy.T.shape


KeyboardInterrupt



In [37]:
neuroviewer.load_dff_df()

loading dff_df


In [ ]:
neurovi

In [77]:
neuroviewer.im
napari.run()

In [9]:
#viewer = napari.Viewer()



viewer1d = napari_plot.ViewerModel1D()
widget = QtViewer(viewer1d)
viewer.window.add_dock_widget(widget, area="bottom", name="Line Widget")

# example data
x = np.arange(0.1, 4, 0.1)
y1 = np.exp(-1.0 * x)
y2 = np.exp(-0.5 * x)

# example variable error bar values
y1err = 0.1 + 0.1 * np.sqrt(x)
y2err = 0.1 + 0.1 * np.sqrt(x / 2)


viewer1d.add_line(np.c_[x, y1], name="Line 1", color="lightblue")
viewer1d.add_centroids(
    np.c_[x, y1 - y1err, y1 + y1err], orientation="vertical", color="lightblue", opacity=0.5, name="Line 1 (errors)"
)
viewer1d.add_line(np.c_[x, y2], name="Line 2", color="orange")
viewer1d.add_centroids(
    np.c_[x, y2 - y2err, y2 + y2err], orientation="vertical", color="orange", opacity=0.5, name="Line 2 (errors)"
)
viewer1d.camera.extent = (-0.1, 4.1, 1.0, -0.3)
viewer1d.axis.x_label = ""
viewer1d.axis.y_label = ""
viewer1d.reset_view()

napari.run()

In [34]:
neuroviewer.add_dff_widget()